# Statistics of ideal mixing
Testing the equilibrium properties of the `SaltSwap` code by exchanging water molecules with water molecules. Non-bonded parameters will be 'updated' to their current values. Therefore, the identities of 'cations' and 'anions' should come in via diffusion.

In [1]:
from simtk import openmm, unit
from simtk.openmm import app
from openmmtools.testsystems import WaterBox
import sys
sys.path.append("../saltswap/")
sys.path.append("../SAMS/")
from calibrate import MCMCSampler
from saltswap import SaltSwap
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

As with other notebooks, the test system will be small box of water.

In [2]:
size = 20.0*unit.angstrom     
temperature = 300*unit.kelvin
pressure = 1*unit.atmospheres
delta_chem = 0*unit.kilojoule_per_mole
wbox = WaterBox(box_edge=size,nonbondedMethod=app.PME)
Nmols = wbox.system.getNumParticles()/3.0

# The effect of shadow work on the acceptance rate.

Initialising the `MCMCSampler` object and changing the ion parameters to the same as water, so that there is no change in energy when salt is 'added'.

In [ ]:
state = MCMCSampler(wbox.system,wbox.topology,wbox.positions)
state.saltswap.cation_parameters = state.saltswap.water_parameters
state.saltswap.anion_parameters = state.saltswap.water_parameters

**INSTANT SWITCHES**

In [ ]:
Nwats = []
Nsalts = []
for block in range(500):
    state.gen_label(saltsteps=20)
    (nwats,nsalt,junk) = state.saltswap.getIdentityCounts()
    Nwats.append(nwats)
    Nsalts.append(nsalt)

In [ ]:
inst_work_add = np.array(state.saltswap.work_add)
inst_work_rm = np.array(state.saltswap.work_rm)
print np.sum(inst_work_add<0)/(1.0*inst_work_add.shape[0])
print np.sum(inst_work_rm<0)/(1.0*inst_work_rm.shape[0])

Viewing the average number of salt pairs as time progresses:

In [ ]:
time = np.arange(len(Nwats))
Nwats = np.array(Nwats)
Nsalts = np.array(Nsalts)

plt.clf()
plt.plot(Nwats/Nmols)
plt.plot(Nsalts/Nmols)
plt.xlabel('Frame')
plt.ylabel('Fractional number')
plt.legend(['Waters','Cations'])
plt.savefig('temp.png',format='png')
print 'Mean fraction of water molecules = {:.3f} +/- {:.3}'.format((Nwats/Nmols).mean(),2*(Nwats/Nmols).std())
print 'Mean fraction of cations = {:.3f} +/- {:.3}'.format((Nsalts/Nmols).mean(),2*(Nsalts/Nmols).std())
print 'Acceptance probability = {:.3f}'.format(state.saltswap.getAcceptanceProbability())
print 'Ratio of variance: std(water)/std(cation) = {:.3f}'.format(Nwats.std()/Nsalts.std())
Image('temp.png')

**NCMC**

Number of perturbation kernels = 512,
Number of propagation kernels = 1

In [ ]:
ncstate = MCMCSampler(wbox.system,wbox.topology,wbox.positions,nkernels = 500,nverlet=1)
ncstate.saltswap.cation_parameters = ncstate.saltswap.water_parameters
ncstate.saltswap.anion_parameters = ncstate.saltswap.water_parameters

In [ ]:
ncNwats = []
ncNsalts = []
for block in range(500):
    ncstate.gen_label(saltsteps=20)
    (nwats,nsalt,junk) = ncstate.saltswap.getIdentityCounts()
    ncNwats.append(nwats)
    ncNsalts.append(nsalt)

In [ ]:
nc_work_add = np.array(ncstate.saltswap.work_add)
nc_work_rm = np.array(ncstate.saltswap.work_rm)
print np.sum(nc_work_add<0)/(1.0*nc_work_add.shape[0])
print np.sum(nc_work_rm<0)/(1.0*nc_work_rm.shape[0])

In [ ]:
time = np.arange(len(ncNwats))
Nwats = np.array(ncNwats)
Nsalts = np.array(ncNsalts)

plt.clf()
plt.plot(Nwats/Nmols)
plt.plot(Nsalts/Nmols)
plt.xlabel('Frame')
plt.ylabel('Fractional number')
plt.legend(['Waters','Cations'])
plt.savefig('temp.png',format='png')
print 'Mean fraction of water molecules = {:.3f} +/- {:.3}'.format((Nwats/Nmols).mean(),2*(Nwats/Nmols).std())
print 'Mean fraction of cations = {:.3f} +/- {:.3}'.format((Nsalts/Nmols).mean(),2*(Nsalts/Nmols).std())
print 'Acceptance probability = {:.3f}'.format(state.saltswap.getAcceptanceProbability())
print 'Ratio of variance: std(water)/std(cation) = {:.3f}'.format(Nwats.std()/Nsalts.std())
Image('temp.png')